In [1]:
# importing required packages
import sys
from pyspark import SparkConf, SparkContext
from math import sqrt

In [6]:
import pandas as pd
import numpy as np
import sys
import logging
from textblob import Word
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import re
import spacy
nlp = spacy.load("en_core_web_sm")
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
stopwords = list(STOP_WORDS)
from heapq import nlargest
import torch
# import tensorflow as tf
# import tensorflow_hub as hub
import numpy as np
import os, sys
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
conf = SparkConf().setMaster("local[*]").setAppName("question3")
sc = SparkContext(conf = conf)

In [3]:
from pyspark.sql import SQLContext, Row 
sqlContext = SQLContext(sc)

## news_articles_data reliance industries

In [86]:
news_data = sqlContext.read.format('csv').options(header="true").load('/Users/praveen/Downloads/isb/term5/FP2/data_files/news_data/reliance_industries_news.csv')

In [87]:
news_data.show(2)

+---+--------------------+---------+-----------+-------------+--------------------+--------------------+--------------------+
|_c0|                date|source-id|source-name|       author|               title|         description|             content|
+---+--------------------+---------+-----------+-------------+--------------------+--------------------+--------------------+
|  0|2021-09-03T04:58:00Z|  reuters|    Reuters|      Reuters|Indian shares hit...|Indian shares sca...|People walk past ...|
|  1|2021-09-03T04:10:00Z|  reuters|    Reuters|Reuters Staff|Indian shares hit...|Indian shares sca...|    By Reuters Staff|
+---+--------------------+---------+-----------+-------------+--------------------+--------------------+--------------------+
only showing top 2 rows



### performing text analytics on spark data frame

In [88]:
from pyspark.sql import functions as F
from pyspark .sql import types as T
from pyspark .sql import Window

In [89]:
def convert_to_str(data):
    return str(data)
convert_to_str_udf = F.udf(convert_to_str,returnType=T.StringType())

In [90]:
news_data = news_data.withColumn("clean_trans",convert_to_str_udf(news_data["content"]))

In [91]:
news_data.show(1)

+---+--------------------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+
|_c0|                date|source-id|source-name| author|               title|         description|             content|         clean_trans|
+---+--------------------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+
|  0|2021-09-03T04:58:00Z|  reuters|    Reuters|Reuters|Indian shares hit...|Indian shares sca...|People walk past ...|People walk past ...|
+---+--------------------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+
only showing top 1 row



In [92]:
def data_clean1(x):
    return re.sub('[^a-zA-Z0-9]', ' ', x)
data_clean1_udf = F.udf(data_clean1,returnType=T.StringType())

In [93]:
news_data = news_data.withColumn("clean_trans",data_clean1_udf(news_data["clean_trans"]))

In [94]:
def data_clean2(x):
    return re.sub(r'\s+', ' ', x)
data_clean2_udf = F.udf(data_clean2,returnType=T.StringType())

In [95]:
news_data = news_data.withColumn("clean_trans",data_clean2_udf(news_data["clean_trans"]))

In [96]:
def nltk_stopword(df):
    stopwords = set(nltk.corpus.stopwords.words('english'))
    stopwords = set([word for word in list(stopwords) if word not in ['not', 'NOT']])
    try:

        words = df.split(" ")
        shortlisted_words = []

        for w in words:
            if w not in stopwords:
                shortlisted_words.append(w)

    except:
        print('An error occurred.')
        shortlisted_words = []
    return ' '.join(shortlisted_words)
nltk_stopword_udf = F.udf(nltk_stopword,returnType=T.StringType())

In [97]:
news_data = news_data.withColumn("clean_trans",nltk_stopword_udf(news_data["clean_trans"]))

In [98]:
def sentiment(x):
    return TextBlob(x).sentiment[0]
sentiment_udf = F.udf(sentiment,returnType=T.StringType())

In [99]:
news_data = news_data.withColumn("sentiment_score",sentiment_udf(news_data["clean_trans"]))

In [100]:
news_data.show(1)

+---+--------------------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
|_c0|                date|source-id|source-name| author|               title|         description|             content|         clean_trans|sentiment_score|
+---+--------------------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
|  0|2021-09-03T04:58:00Z|  reuters|    Reuters|Reuters|Indian shares hit...|Indian shares sca...|People walk past ...|People walk past ...|          -0.25|
+---+--------------------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
only showing top 1 row



In [101]:
def Date_function(x):
    x = str(x)
    try:
        return x.split('T')[0]
    except:
        return ''
Date_function_udf = F.udf(Date_function,returnType=T.StringType())

In [102]:
news_data = news_data.withColumn("Date",Date_function_udf(news_data["date"]))

In [106]:
news_data = news_data.withColumn("sentiment_score",news_data.sentiment_score.cast(T.FloatType()))

In [107]:
news_data.show(1)

+---+----------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
|_c0|      Date|source-id|source-name| author|               title|         description|             content|         clean_trans|sentiment_score|
+---+----------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
|  0|2021-09-03|  reuters|    Reuters|Reuters|Indian shares hit...|Indian shares sca...|People walk past ...|People walk past ...|          -0.25|
+---+----------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
only showing top 1 row



In [114]:

sentiment_data = news_data.groupBy("Date").agg(F.mean("sentiment_score").alias("sentiment_score")) 

In [115]:
sentiment_data.show(2)

+----------+--------------------+
|      Date|     sentiment_score|
+----------+--------------------+
|2021-08-30|-0.13750000298023224|
|2021-09-06| 0.02014266022227027|
+----------+--------------------+
only showing top 2 rows



## merging news data sentiment with nifty data 

In [46]:
nifty_data = sqlContext.read.format('csv').options(header="true").load('/Users/praveen/Downloads/isb/term5/FP2/data_files/fianance_data/share_data/nifty50_share_closing_price.csv')

In [47]:
nifty_data = nifty_data.withColumn("nifty50_close",nifty_data['Close'])
nifty_data = nifty_data.drop('Close')

In [48]:
nifty_data.show(2)

+----------+----------------+
|      Date|   nifty50_close|
+----------+----------------+
|2021-07-28| 15709.400390625|
|2021-07-29|15778.4501953125|
+----------+----------------+
only showing top 2 rows



In [120]:
share_data = sqlContext.read.format('csv').options(header="true").load('/Users/praveen/Downloads/isb/term5/FP2/data_files/fianance_data/share_data/infosys_share_closing_price.csv')

In [121]:
share_data = share_data.withColumn("share_close",share_data['Close'])
share_data = share_data.drop('Close')

In [122]:
share_data.show(2)

+----------+-------+------------------+
|      Date| Volume|       share_close|
+----------+-------+------------------+
|2021-07-28|4686788|1605.5999755859375|
|2021-07-29|4660873|            1617.0|
+----------+-------+------------------+
only showing top 2 rows



In [123]:
share_data = nifty_data.join(share_data,nifty_data.Date ==  share_data.Date,"inner").drop(share_data.Date)

In [124]:
share_data.show(2) 

+----------+----------------+-------+------------------+
|      Date|   nifty50_close| Volume|       share_close|
+----------+----------------+-------+------------------+
|2021-07-28| 15709.400390625|4686788|1605.5999755859375|
|2021-07-29|15778.4501953125|4660873|            1617.0|
+----------+----------------+-------+------------------+
only showing top 2 rows



In [125]:
news_data.show(1)

+---+----------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
|_c0|      Date|source-id|source-name| author|               title|         description|             content|         clean_trans|sentiment_score|
+---+----------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
|  0|2021-09-03|  reuters|    Reuters|Reuters|Indian shares hit...|Indian shares sca...|People walk past ...|People walk past ...|          -0.25|
+---+----------+---------+-----------+-------+--------------------+--------------------+--------------------+--------------------+---------------+
only showing top 1 row



In [126]:
share_data = share_data.join(sentiment_data,sentiment_data.Date ==  share_data.Date,"left").drop(sentiment_data.Date)

In [127]:
share_data.show(1)

+----------+---------------+-------+------------------+---------------+
|      Date|  nifty50_close| Volume|       share_close|sentiment_score|
+----------+---------------+-------+------------------+---------------+
|2021-07-28|15709.400390625|4686788|1605.5999755859375|           null|
+----------+---------------+-------+------------------+---------------+
only showing top 1 row



In [128]:
share_data = share_data.na.fill(value=0,subset=["sentiment_score"])

In [131]:
share_data.show(20)

+----------+----------------+-------+------------------+--------------------+
|      Date|   nifty50_close| Volume|       share_close|     sentiment_score|
+----------+----------------+-------+------------------+--------------------+
|2021-07-28| 15709.400390625|4686788|1605.5999755859375|                 0.0|
|2021-07-29|15778.4501953125|4660873|            1617.0|                 0.0|
|2021-07-30|15763.0498046875|6234257|            1610.5|                 0.0|
|2021-08-02| 15885.150390625|5553389| 1631.550048828125|                 0.0|
|2021-08-03|        16130.75|5513668| 1655.199951171875|                 0.0|
|2021-08-04|16258.7998046875|6740391|           1649.25|                 0.0|
|2021-08-05| 16294.599609375|5420536| 1653.550048828125|                 0.0|
|2021-08-06|16238.2001953125|3754855| 1650.199951171875|                 0.0|
|2021-08-09|        16258.25|5018477| 1663.300048828125|                 0.0|
|2021-08-10| 16280.099609375|6846517|           1677.25|        

In [132]:
sqlContext.registerDataFrameAsTable(share_data,'share_data')